In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def hello_world():
    return 'Hello, World!'

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home</title>
</head>
<body>
    <h1>Welcome to the Home Page</h1>
    <p>This is the home page of our Flask app.</p>
    <a href="{{ url_for('about') }}">Go to About Page</a>
</body>
</html>
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>About</title>
</head>
<body>
    <h1>About Us</h1>
    <p>This is the about page of our Flask app.</p>
    <a href="{{ url_for('home') }}">Go to Home Page</a>
</body>
</html>

from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    return render_template('home.html')

@app.route('/about')
def about():
    return render_template('about.html')

if __name__ == '__main__':
    app.run(debug=True)



In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

# Define a route that accepts a parameter in the URL
@app.route('/user/<username>')
def user_profile(username):
    # Render a template and pass the parameter to it
    return render_template('user_profile.html', username=username)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from flask import Flask, render_template, request

app = Flask(__name__)

# Define a route for the form
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Get the user input from the form
        user_input = request.form['user_input']
        # Render a template and pass the user input to it
        return render_template('result.html', user_input=user_input)
    # If it's a GET request, just render the form template
    return render_template('form.html')

if __name__ == '__main__':
    app.run(debug=True)

    
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Input Form</title>
</head>
<body>
    <h1>Enter your input:</h1>
    <form method="post">
        <input type="text" name="user_input">
        <button type="submit">Submit</button>
    </form>
</body>
</html>



<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Result</title>
</head>
<body>
    <h1>Your input:</h1>
    <p>{{ user_input }}</p>
</body>
</html>


In [ ]:
from flask import Flask, render_template, request, session, redirect, url_for
from datetime import timedelta

app = Flask(__name__)

# Set the secret key for sessions
app.secret_key = 'your_secret_key'
# Set session lifetime
app.permanent_session_lifetime = timedelta(minutes=5)

# Define a route for the form
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        # Get the user input from the form
        user_input = request.form['user_input']
        # Store user input in session
        session['user_input'] = user_input
        # Redirect to the result page
        return redirect(url_for('result'))
    # If it's a GET request, just render the form template
    return render_template('form.html')

# Define a route for displaying user-specific data
@app.route('/result')
def result():
    # Retrieve user input from session
    user_input = session.get('user_input')
    # If user input exists in session, render the result template
    if user_input:
        return render_template('result.html', user_input=user_input)
    # If user input does not exist in session, redirect to the form page
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import os

app = Flask(__name__)

# Define the upload folder
UPLOAD_FOLDER = 'uploads'
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Ensure the upload folder exists
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Define a route for the file upload form
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the POST request has the file part
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        # If the user does not select a file, the browser submits an empty file without a filename
        if file.filename == '':
            return redirect(request.url)
        if file:
            # Save the uploaded file to the upload folder
            filename = file.filename
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('display_file', filename=filename))
    # If it's a GET request, just render the upload form template
    return render_template('upload_form.html')

# Define a route for displaying the uploaded file
@app.route('/uploads/<filename>')
def display_file(filename):
    return render_template('display_file.html', filename=filename)

if __name__ == '__main__':
    app.run(debug=True)

    
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Upload File</title>
</head>
<body>
    <h1>Upload File</h1>
    <form method="post" enctype="multipart/form-data">
        <input type="file" name="file">
        <button type="submit">Upload</button>
    </form>
</body>
</html>



<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Display File</title>
</head>
<body>
    <h1>Uploaded File</h1>
    <img src="{{ url_for('static', filename='uploads/' + filename) }}" alt="Uploaded File">
    <p>Filename: {{ filename }}</p>
</body>
</html>


In [ ]:
from flask import Flask, render_template, request, redirect, url_for
import sqlite3

app = Flask(__name__)

# Function to create a connection to the SQLite database
def get_db_connection():
    conn = sqlite3.connect('items.db')
    conn.row_factory = sqlite3.Row
    return conn

# Function to initialize the database
def init_db():
    conn = get_db_connection()
    with app.open_resource('schema.sql', mode='r') as f:
        conn.cursor().executescript(f.read())
    conn.commit()
    conn.close()

# Initialize the database
init_db()

# Define a route for the home page to display the list of items
@app.route('/')
def index():
    conn = get_db_connection()
    items = conn.execute('SELECT * FROM items').fetchall()
    conn.close()
    return render_template('index.html', items=items)

# Define a route for adding a new item
@app.route('/add', methods=['POST'])
def add_item():
    if request.method == 'POST':
        item_name = request.form['item_name']
        conn = get_db_connection()
        conn.execute('INSERT INTO items (name) VALUES (?)', (item_name,))
        conn.commit()
        conn.close()
    return redirect(url_for('index'))

# Define a route for deleting an item
@app.route('/delete/<int:item_id>')
def delete_item(item_id):
    conn = get_db_connection()
    conn.execute('DELETE FROM items WHERE id = ?', (item_id,))
    conn.commit()
    conn.close()
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)
    
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Items List</title>
</head>
<body>
    <h1>Items List</h1>
    <ul>
        {% for item in items %}
        <li>{{ item['name'] }} <a href="/delete/{{ item['id'] }}">Delete</a></li>
        {% endfor %}
    </ul>
    <h2>Add New Item</h2>
    <form action="/add" method="post">
        <input type="text" name="item_name" required>
        <button type="submit">Add Item</button>
    </form>
</body>
</html>


CREATE TABLE IF NOT EXISTS items (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL
);



In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# Flask-Login setup
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'

# User class/model
class User(UserMixin):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password

# Mock user database
users = []

# Helper function to get user by username
def get_user(username):
    for user in users:
        if user.username == username:
            return user
    return None

# Login manager user loader callback
@login_manager.user_loader
def load_user(user_id):
    return User.get(user_id)

# Routes
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        user = get_user(username)
        if user and check_password_hash(user.password, password):
            login_user(user)
            flash('Logged in successfully!', 'success')
            return redirect(url_for('profile'))
        else:
            flash('Invalid username or password', 'error')
    return render_template('login.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if get_user(username):
            flash('Username already exists', 'error')
        else:
            hashed_password = generate_password_hash(password)
            user = User(len(users) + 1, username, hashed_password)
            users.append(user)
            flash('Registration successful! Please log in.', 'success')
            return redirect(url_for('login'))
    return render_template('register.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    flash('Logged out successfully!', 'success')
    return redirect(url_for('login'))

@app.route('/profile')
@login_required
def profile():
    return render_template('profile.html', username=current_user.username)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# Sample data - list of books
books = [
    {"id": 1, "title": "Book 1", "author": "Author 1"},
    {"id": 2, "title": "Book 2", "author": "Author 2"},
    {"id": 3, "title": "Book 3", "author": "Author 3"}
]

# Route to get all books
@app.route('/books', methods=['GET'])
def get_books():
    return jsonify(books)

# Route to get a specific book by ID
@app.route('/books/<int:book_id>', methods=['GET'])
def get_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if book:
        return jsonify(book)
    return jsonify({"error": "Book not found"}), 404

# Route to add a new book
@app.route('/books', methods=['POST'])
def add_book():
    new_book = request.json
    new_book['id'] = len(books) + 1
    books.append(new_book)
    return jsonify({"message": "Book added successfully", "book": new_book}), 201

# Route to update an existing book
@app.route('/books/<int:book_id>', methods=['PUT'])
def update_book(book_id):
    book = next((book for book in books if book['id'] == book_id), None)
    if not book:
        return jsonify({"error": "Book not found"}), 404
    updated_data = request.json
    book.update(updated_data)
    return jsonify({"message": "Book updated successfully", "book": book})

# Route to delete a book
@app.route('/books/<int:book_id>', methods=['DELETE'])
def delete_book(book_id):
    global books
    books = [book for book in books if book['id'] != book_id]
    return jsonify({"message": "Book deleted successfully"})

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home Page</title>
</head>
<body>
    <h1>Welcome to the Home Page</h1>
</body>
</html>


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>404 Not Found</title>
</head>
<body>
    <h1>404 - Not Found</h1>
    <p>The page you are looking for does not exist.</p>
</body>
</html>


<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>500 Internal Server Error</title>
</head>
<body>
    <h1>500 - Internal Server Error</h1>
    <p>There was an internal server error. Please try again later.</p>
</body>
</html>


In [ ]:
<!-- index.html -->
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chat Application</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.1.3/socket.io.js" integrity="sha384-xz6CvRNAXULsEPoOJ1Jpg+qV8ctVNSQ0nIvp5VNWuSsGogYjgFQf3vvvZL9FS0W/" crossorigin="anonymous"></script>
    <script>
        var socket = io.connect('http://' + document.domain + ':' + location.port);
        socket.on('message', function(msg) {
            var li = document.createElement('li');
            li.textContent = msg;
            document.getElementById('messages').appendChild(li);
        });
        function sendMessage() {
            var msg = document.getElementById('message').value;
            socket.emit('message', msg);
            document.getElementById('message').value = '';
        }
    </script>
</head>
<body>
    <h1>Chat Application</h1>
    <ul id="messages"></ul>
    <input type="text" id="message" placeholder="Type your message">
    <button onclick="sendMessage()">Send</button>
</body>
</html>


In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit
import time
import threading

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

# Function to periodically update data and emit it to clients
def update_data():
    while True:
        # Generate some data (for demonstration purposes)
        data = {'value': time.strftime('%H:%M:%S')}
        # Emit the data to all connected clients
        socketio.emit('update', data)
        # Sleep for 1 second
        time.sleep(1)

# Route for the main page
@app.route('/')
def index():
    return render_template('index.html')

# Event handler for when a client connects
@socketio.on('connect')
def handle_connect():
    print('Client connected')

# Event handler for when a client disconnects
@socketio.on('disconnect')
def handle_disconnect():
    print('Client disconnected')

if __name__ == '__main__':
    # Start a separate thread to periodically update data
    threading.Thread(target=update_data, daemon=True).start()
    # Run the Flask app with Flask-SocketIO
    socketio.run(app, debug=True)

    
    <!-- index.html -->
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Real-Time Data Updates</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.1.3/socket.io.js" integrity="sha384-xz6CvRNAXULsEPoOJ1Jpg+qV8ctVNSQ0nIvp5VNWuSsGogYjgFQf3vvvZL9FS0W/" crossorigin="anonymous"></script>
    <script>
        var socket = io.connect('http://' + document.domain + ':' + location.port);
        socket.on('update', function(data) {
            document.getElementById('data').textContent = data.value;
        });
    </script>
</head>
<body>
    <h1>Real-Time Data Updates</h1>
    <p>Data value: <span id="data"></span></p>
</body>
</html>


In [ ]:
from flask import Flask, render_template
from flask_socketio import SocketIO, emit

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
socketio = SocketIO(app)

# Route for the main page
@app.route('/')
def index():
    return render_template('index.html')

# Event handler for when a client connects
@socketio.on('connect')
def handle_connect():
    print('Client connected')

# Event handler for when a client disconnects
@socketio.on('disconnect')
def handle_disconnect():
    print('Client disconnected')

# Route to trigger a notification
@app.route('/notify')
def notify():
    # Emit a notification to all connected clients
    socketio.emit('notification', {'message': 'New update available!'})
    return 'Notification sent'

if __name__ == '__main__':
    # Run the Flask app with Flask-SocketIO
    socketio.run(app, debug=True)

    
    <!-- index.html -->
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Notifications</title>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/socket.io/4.1.3/socket.io.js" integrity="sha384-xz6CvRNAXULsEPoOJ1Jpg+qV8ctVNSQ0nIvp5VNWuSsGogYjgFQf3vvvZL9FS0W/" crossorigin="anonymous"></script>
    <script>
        var socket = io.connect('http://' + document.domain + ':' + location.port);
        socket.on('notification', function(data) {
            alert(data.message);
        });
    </script>
</head>
<body>
    <h1>Notifications</h1>
    <button onclick="notify()">Trigger Notification</button>
    <script>
        function notify() {
            fetch('/notify');
        }
    </script>
</body>
</html>
